# Detect A-E Mismatch

## Train model to predict answer using Question + Explanation

Approach:
    - Question --> gpt2 --> Q_embeddings
        Explanations --> gpt2 --> E_embeddings
    - Merge Q+E --> Send to Classifier

In [1]:
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel
import pandas as pd
import math
import torch
import numpy as np

# https://github.com/Meelfy/pytorch_pretrained_BERT#usage

model = OpenAIGPTModel.from_pretrained('openai-gpt')
model.eval()
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')


ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


['hello</w>']

In [2]:
data = pd.read_csv('Post_processing.csv', index_col=0)

In [99]:
#%%time
import os


SAVEPATH = r'text_vectors\\'
def get_encoding(row,idx, col = None, encoded = True):
    if col == None:
        col = ['exp', 'qstr']
    for j in col:
        sent = row[j]
        sent_tok = tokenizer.tokenize(sent)
        sent_tok_idx = tokenizer.convert_tokens_to_ids(sent_tok)
        tok_tensor = torch.tensor([sent_tok_idx])
        if encoded == True:
            encoded = model(tok_tensor)
            encoded = encoded.detach().numpy()
        else:
            encoded = sent_tok_idx
        np.save(f'{SAVEPATH}{j}\\{j}_{str(idx)}', encoded)
    return 'Done'

#data['exp_enc'] = data['exp']
#data['qstr_enc'] = data['qstr']


for idx,row in data.iterrows():
    get_encoding(row,idx,['ans','pred'],False)
# data['qstr_enc'].apply((lambda x: get_encoding(x)))



data

,ans,exp,pred,qstr,label,comment,img_enc,qstr_enc,exp_enc
363036001,cake,there is a pastry base which has been adorned ...,cake,what are they eating,5,NaN,"[1.07310325e-05, 6.549826e-05, 3.273e-05, 2.81...","[[[0.22124447, -0.44978356, -0.13344349, 0.015...","[[[0.88021237, -0.19273034, -0.06036195, -0.09..."
426070000,couch,it is a bed,couch,what is the man sitting on,2,NaN,"[7.4897216e-06, 3.1976388e-06, 2.6436192e-06, ...","[[[0.22124447, -0.44978356, -0.13344349, 0.015...","[[[0.49296755, 0.0056634257, -0.14168072, -0.4..."
75560012,yes,there are clouds in the sky,yes,is it cloudy,5,NaN,"[5.4029704e-05, 2.6568987e-05, 3.3543776e-05, ...","[[[-0.047832504, 0.069201425, 0.013050009, -0....","[[[0.88021237, -0.19273034, -0.06036195, -0.09..."
253810003,yes,it is yellow,no,do the bananas look fresh,2,NaN,"[1.3661576e-05, 6.8835975e-06, 1.6233305e-05, ...","[[[-0.32399142, -0.29960737, 0.26913592, -0.45...","[[[0.4929689, 0.005664519, -0.14168045, -0.468..."
206878001,yes,it is a big portion of a plated,yes,does this food look sweet,1,NaN,"[3.583257e-05, 1.6358184e-05, 7.609522e-05, 4....","[[[-0.36285457, -0.1840965, 0.2716377, -0.4769...","[[[0.49296755, 0.0056634257, -0.14168072, -0.4..."
93236000,no,the train is sitting at the station,no,is the train moving,5,NaN,"[0.00024376233, 0.00016615825, 0.000120588506,...","[[[-0.047832914, 0.069202095, 0.013049648, -0....","[[[0.251859, -0.13790593, 0.10117143, -0.18055..."
278601006,yes,it is parked,no,are a majority of the trucks windows down,2,NaN,"[0.00038697233, 0.00012865967, 8.070885e-05, 6...","[[[0.07479936, 0.1508446, 0.10458157, -0.44666...","[[[0.4929689, 0.005664519, -0.14168045, -0.468..."
126995004,tennis,the man is holding a tennis racket,tennis,what game are they playing,5,NaN,"[0.0003120446, 0.00031408554, 0.00033154766, 0...","[[[0.22124447, -0.44978356, -0.13344349, 0.015...","[[[0.251859, -0.13790593, 0.10117143, -0.18055..."
192168002,no,the man is wearing a suit and a woman to weari...,no,is the man dressed for a sporting event,1,NaN,"[7.694638e-05, 1.5928821e-05, 2.6912421e-05, 4...","[[[-0.047832914, 0.069202095, 0.013049648, -0....","[[[0.251859, -0.13790593, 0.10117143, -0.18055..."
197022001,no,there is a slice of yellow substance with hole...,no,do these pizzas have the same toppings,2,NaN,"[0.00012107052, 0.00016793661, 0.00015035561, ...","[[[-0.32399142, -0.29960737, 0.26913592, -0.45...","[[[0.88021237, -0.19273034, -0.06036195, -0.09..."


In [66]:
get_encoding(data.loc[291404001].exp).detach().numpy().size

5


4608

In [96]:
data = pd.read_pickle('Post_Processing_with_Encoding.pkl')
t = data['ans'].sample().item()
r = tokenizer.tokenize(t)
tokenizer.convert_tokens_to_ids(r), t, r

# Sometimes words exist of 2 tokens,
# We will presume it got the answer correct if it can guess the first one

([29506, 12370], 'skateboarding', ['skate', 'boarding</w>'])

In [105]:
len(data.pred.unique())

126

## Create Train Test Pickles:

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_pickle('Post_Processing_with_Encoding_ans_pred.pkl')

data_train, data_test = train_test_split(data,stratify = data.label, random_state=42)




In [21]:
print('test:')
print(data_test.groupby('label').count()/len(data_test))

print('train:')
print(data_train.groupby('label').count()/len(data_train))

data_test.to_pickle('PPEAP_test.pkl')
data_train.to_pickle('PPEAP_train.pkl')

test:
            ans       exp      pred      qstr   comment   img_enc   exp_enc  \
label                                                                         
1      0.097561  0.097561  0.097561  0.097561  0.018293  0.097561  0.097561   
2      0.414634  0.414634  0.414634  0.414634  0.081301  0.414634  0.414634   
3      0.018293  0.018293  0.018293  0.018293  0.006098  0.018293  0.018293   
4      0.046748  0.046748  0.046748  0.046748  0.030488  0.046748  0.046748   
5      0.418699  0.418699  0.418699  0.418699  0.024390  0.418699  0.418699   
6      0.004065  0.004065  0.004065  0.004065  0.004065  0.004065  0.004065   

       qstr_enc   ans_enc  pred_enc  
label                                
1      0.097561  0.097561  0.097561  
2      0.414634  0.414634  0.414634  
3      0.018293  0.018293  0.018293  
4      0.046748  0.046748  0.046748  
5      0.418699  0.418699  0.418699  
6      0.004065  0.004065  0.004065  
train:
            ans       exp      pred      qstr   co

# Model

### Preprocessing

In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def pad_exp(exp):
    
    
    s = exp.shape
    out = np.pad(exp, ((0,0),(0,22-s[1]),(0,0)),'constant', constant_values = 0)
    
    return out

#fake = np.random.rand(918,22,768)

# ones, zeros = data.label.value_counts()

# ones/(ones+zeros)

# data.label.value_counts()

In [23]:

# data = pd.read_pickle('Post_Processing_with_Encoding_ans_pred.pkl')
ohe_len = len(data.pred.unique())

enc = OneHotEncoder().fit(data.pred.to_numpy().reshape(-1,1))


## Preprocessing
def preprocess_dat(data):

    data_train_test = data[(data.label == 5)]
    data_eval_ea = data[(data.label == 4) | (data.label == 5)]
    data_eval_ea.label[data_eval_ea.label == 4] = 1
    data_eval_ea.label[data_eval_ea.label == 5] = 0
    data_train_test.exp_enc = data_train_test.exp_enc.apply(pad_exp)
    data_train_test.qstr_enc = data_train_test.qstr_enc.apply(pad_exp)
    
    return data_train_test, data_eval_ea

train = pd.read_pickle('PPEAP_train.pkl')
test = pd.read_pickle('PPEAP_test.pkl')


train, eval_train = preprocess_dat(train)
test, eval_test = preprocess_dat(test)


#data.ans_enc.apply(lambda x: x[0])

# Use pd dummies instead of GPT token id's since it is less hassle.


#data.label= np.random.choice([0,1],len(data.label))



c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c

In [3]:
# enc.transform(data.ans.to_numpy().reshape(-1,1))
# pd.Series(data.ans.unique()).diff(pd.Series(data.pred.unique()))


### Model building

In [24]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hidden_size = [64,64], amount_layers = 2, dropout = []):

    input_qstr = keras.Input(shape=(22,768))
    flatten_qstr = layers.Flatten()(input_qstr)

    input_exp = keras.Input(shape=(22,768))
    flatten_exp = layers.Flatten()(input_exp)

    concat = layers.concatenate(([flatten_qstr, flatten_exp]))

    d = layers.Dense(64,activation = 'relu')(concat)
    for l in dropout:
        
    d = layers.Dense(64, activation = 'relu')(d)

    output = layers.Dense(ohe_len, activation = 'softmax')(d)

    model = keras.Model([input_qstr,input_exp],output)
    model.summary()
    model.compile(loss=keras.losses.CategoricalCrossentropy(),optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 22, 768)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 22, 768)]    0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 16896)        0           input_1[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 16896)        0           input_2[0][0]                    
______________________________________________________________________________________________

## Experiment with different model architectures

### Training the answer guesser

In [25]:
from sklearn.model_selection import train_test_split

#train, test = train_test_split(data_train_test, random_state=42)


model.fit([np.vstack(train['qstr_enc'].to_numpy()),
          np.vstack(train['exp_enc'].to_numpy())],
          enc.transform(train.pred.to_numpy().reshape(-1,1)).todense(),
          batch_size=32,
          epochs=20)

model.save('AE_Mismatch_model.tf')

Epoch 1/20
20/20 [==============================] - 0s 6ms/step - loss: 3.1910 - accuracy: 0.4065
Epoch 2/20
20/20 [==============================] - 0s 5ms/step - loss: 1.4006 - accuracy: 0.6774
Epoch 3/20
20/20 [==============================] - 0s 5ms/step - loss: 0.7899 - accuracy: 0.8065
Epoch 4/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4596 - accuracy: 0.8855
Epoch 5/20
20/20 [==============================] - 0s 4ms/step - loss: 0.2673 - accuracy: 0.9403
Epoch 6/20
20/20 [==============================] - 0s 5ms/step - loss: 0.1422 - accuracy: 0.9645
Epoch 7/20
20/20 [==============================] - 0s 5ms/step - loss: 0.0930 - accuracy: 0.9726
Epoch 8/20
20/20 [==============================] - 0s 5ms/step - loss: 0.0738 - accuracy: 0.9758
Epoch 9/20
20/20 [==============================] - 0s 5ms/step - loss: 0.0825 - accuracy: 0.9758
Epoch 10/20
20/20 [==============================] - 0s 5ms/step - loss: 0.0439 - accuracy: 0.9903
Epoch 11/20
20/20 [

In [26]:
#evaluating:
from sklearn.metrics import f1_score
print(model.evaluate([np.vstack(test['qstr_enc'].to_numpy()),
          np.vstack(test['exp_enc'].to_numpy())],
          enc.transform(test.pred.to_numpy().reshape(-1,1)).todense()),  model.metrics_names)

predict = model.predict([np.vstack(test['qstr_enc'].to_numpy()),
          np.vstack(test['exp_enc'].to_numpy())])

np.hstack([enc.inverse_transform(predict), test.pred.to_numpy().reshape(-1,1)])

f1_score(enc.inverse_transform(predict), test.pred.to_numpy().reshape(-1,1),average = 'micro')


7/7 [==============================] - 0s 3ms/step - loss: 1.9305 - accuracy: 0.7282
[1.93052077293396, 0.7281553149223328] ['loss', 'accuracy']


0.7281553398058251

## Part 2: predicting the label using pred and pred_pred haha

We have a model that predicts pred from qstr & exp. We will save this prediction in a row called pred_pred, and then compare whether a mismatch (pred_pred!=pred) occurs more often with label 0 or label 1.



In [28]:
data = pd.read_pickle('Post_Processing_with_Encoding_ans_pred.pkl')

data_eval_ea = data[(data.label == 4) | (data.label == 5)]
data_eval_ea.label[data_eval_ea.label == 4] = 1
data_eval_ea.label[data_eval_ea.label == 5] = 0

train_ea, test_ea = train_test_split(data_eval_ea, random_state=42)

# WERKT NIET
# def predict_row(row,model,encoder):
#     exp = np.vstack(row.exp_enc.apply(pad_exp).to_numpy())
#     qstr = np.vstack(row.qstr_enc.apply(pad_exp).to_numpy())
    
#     pred_enc = model.predict([exp,qstr])
#     pred = enc.inverse_transform(pred_enc)
#     return pred

# samp = data_eval_ea.sample()
# predict_row(samp,model,enc) == samp.pred.to_numpy()



# data_eval_ea['pred_pred'] = predict_row(data_eval_ea,model,enc)
#data_eval_ea['pred_ae'] = data_eval_ea['pred_ae']
lab0 = data_eval_ea[data_eval_ea.label==0]
lab1 = data_eval_ea[data_eval_ea.label==1]

# from sklearn.metrics import accuracy_score, classification_report
# 'no_ae:',accuracy_score(lab0.pred.to_numpy().reshape(-1,1), lab0.pred_pred.to_numpy().reshape(-1,1)) , \
# 'ae:', accuracy_score(lab1.pred.to_numpy().reshape(-1,1), lab1.pred_pred.to_numpy().reshape(-1,1))

#classification_report(data_eval_ea.pred, data_eval_ea.pred_pred)
#Note: it just guesses yes most of the time

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [29]:
data_eval_ea.exp_enc = data_eval_ea.exp_enc.apply(pad_exp)
data_eval_ea.qstr_enc = data_eval_ea.qstr_enc.apply(pad_exp)

predict = model.predict([np.vstack(data_eval_ea['qstr_enc'].to_numpy()),
          np.vstack(data_eval_ea['exp_enc'].to_numpy())])
                         

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [30]:
data_eval_ea['pred_pred'] = enc.inverse_transform(predict)

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
pred = data_eval_ea[data_eval_ea.label==1].pred
pred_pred = data_eval_ea[data_eval_ea.label==1].pred_pred

In [32]:
print(accuracy_score(pred,pred_pred)) #36% accuracy 
data_eval_ea[(data_eval_ea.label == 1)& (data_eval_ea.pred==data_eval_ea.pred_pred)]
# Allmost all of the cases where the model could predict the true answer was yes/no questions when the 
# a-e mismatch label was assigned to the explanation

0.41304347826086957


,ans,exp,pred,qstr,label,comment,img_enc,exp_enc,qstr_enc,ans_enc,pred_enc,pred_pred
366421000,no,the window is angled outwards,no,is the window open,1,NaN,"[3.4171422e-05, 4.807944e-05, 4.6459136e-05, 4...","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[664],[664],no
440937002,yes,it has a touch screen and icons,no,is this a smartphone,1,NaN,"[0.00015047418, 0.00014376432, 7.855466e-05, 0...","[[[0.49296755, 0.0056634257, -0.14168072, -0.4...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[685],[664],no
168231003,yes,there is snow,yes,does he like the outdoors,1,Subjectief,"[0.00027916086, 0.00013961525, 6.7289126e-05, ...","[[[0.88021314, -0.192732, -0.060362242, -0.096...","[[[-0.36285457, -0.1840965, 0.2716377, -0.4769...",[685],[685],yes
487607003,no,it is in color and the person is wearing moder...,no,is this a new picture,1,kan ook 2,"[0.00027826228, 8.492977e-05, 0.0005238845, 0....","[[[0.49296755, 0.0056634257, -0.14168072, -0.4...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[664],[664],no
268641000,yes,it is very old and faded,no,does the red fire hydrant need new paint,1,NaN,"[0.00047980415, 0.0017087816, 0.000366393, 0.0...","[[[0.49296755, 0.0056634257, -0.14168072, -0.4...","[[[-0.36285457, -0.1840965, 0.2716377, -0.4769...",[685],[664],no
10995004,yes,the window is angled outwards,no,is the window open,1,kan ook 3,"[3.5225374e-05, 4.865044e-05, 6.683977e-05, 4....","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[685],[664],no
486087000,no,the window is angled outwards,no,is the window open,1,kan ook 2,"[0.000112978705, 0.00026586663, 0.00013565231,...","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[664],[664],no
49005014,yes,the players are wearing professional jersies,no,is this a professional game,1,NaN,"[1.9470324e-06, 4.4734193e-07, 3.498057e-06, 1...","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[685],[664],no
491481016,yes,the sky is gray and there are no clouds visible,yes,is it cloudy,1,"kan ook 2, wel deels goed","[5.819737e-05, 2.731714e-05, 1.7609038e-05, 7....","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[-0.047832504, 0.069201425, 0.013050009, -0....",[685],[685],yes
111062001,yes,the floor is covered in trash,yes,does this room look neat and tidy,1,kan ook 2,"[2.3202387e-05, 2.4946612e-05, 3.0155365e-05, ...","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[-0.36285457, -0.1840965, 0.2716377, -0.4769...",[685],[685],yes


In [33]:
data_eval_ea[(data_eval_ea.label == 1)& (data_eval_ea.pred!=data_eval_ea.pred_pred)]

,ans,exp,pred,qstr,label,comment,img_enc,exp_enc,qstr_enc,ans_enc,pred_enc,pred_pred
372070010,yes,it is stopped at the station to let people on ...,yes,is the train moving,1,NaN,"[8.517646e-05, 9.706394e-05, 6.9598675e-05, 6....","[[[0.49296755, 0.0056634257, -0.14168072, -0.4...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[685],[685],no
432378003,no,it is in black and white and the old style is old,yes,is this a modern picture,1,NaN,"[0.00025608542, 0.00020047021, 0.0002119429, 0...","[[[0.49296755, 0.0056634257, -0.14168072, -0.4...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[664],[685],no
563040007,no,it is parked in a parking lot,no,should the black car be parked there,1,NaN,"[9.433718e-05, 8.376524e-05, 5.6301393e-05, 7....","[[[0.49296755, 0.0056634257, -0.14168072, -0.4...","[[[-0.20759985, -0.2005973, -0.023269953, -0.0...",[664],[664],yes
518438001,stop,the light is green,stop,should the car go or stop,1,NaN,"[3.613428e-06, 3.5851579e-06, 6.892883e-07, 8....","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[-0.20759985, -0.2005973, -0.023269953, -0.0...",[1296],[1296],yes
579697002,china,the writing on the signs is in english,china,what country is this,1,NaN,"[2.1725886e-05, 8.458778e-05, 5.332117e-05, 9....","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[0.22124447, -0.44978356, -0.13344349, 0.015...",[8898],[8898],skateboarding
178748001,electric,the burners are clearly gas burners,electric,is the stove gas or electric,1,NaN,"[9.014874e-05, 0.00018870331, 8.383712e-05, 7....","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[7641],[7641],yes
148531000,no,it is a stealth plane with an older design,yes,is this a modern plane,1,kan ook 2 zijn,"[2.4011177e-05, 1.8703811e-05, 5.3537184e-05, ...","[[[0.49296755, 0.0056634257, -0.14168072, -0.4...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[664],[685],no
332916003,yes,the sign on the sign is not in english,yes,is this in america,1,NaN,"[3.4645498e-05, 2.0619795e-05, 2.897833e-05, 0...","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[-0.047832914, 0.069202095, 0.013049648, -0....",[685],[685],no
385770001,yes,the room is dark and there is no sunlight,yes,is it daytime,1,Kan ook 2 zijn. misschien definities nog iets ...,"[0.00017555842, 0.00021418398, 0.0001182133, 0...","[[[0.251859, -0.13790593, 0.10117143, -0.18055...","[[[-0.047832504, 0.069201425, 0.013050009, -0....",[685],[685],no
441218002,skiing,he is on a snowboard on the snow,skiing,what is the person doing,1,NaN,"[1.3099012e-05, 3.4431578e-05, 0.00011822911, ...","[[[0.11647998, 0.0899214, -0.16073456, -0.5264...","[[[0.22124447, -0.44978356, -0.13344349, 0.015...",[24979],[24979],snowboarding


In [34]:

# When the answer is guessed correctly, how often is it a yes/no question? I suspect this is higher for the a-e category
data_eval_ea[(data_eval_ea.pred==data_eval_ea.pred_pred) &\
             (data_eval_ea.pred_pred.isin(['yes','no']))]\
             .groupby('label').size() / data_eval_ea[(data_eval_ea.pred==data_eval_ea.pred_pred)].groupby('label').size() 

label
0    0.295189
1    1.000000
dtype: float64

In [35]:
#Do the check for test to see if it is also true for values the model hasn't seen yet
test_predict = model.predict([np.vstack(test['qstr_enc'].to_numpy()),
          np.vstack(test['exp_enc'].to_numpy())])
test['pred_pred'] = enc.inverse_transform(test_predict)

In [36]:

# When the answer is guessed correctly, how often is it a yes/no question? I suspect this is higher for the a-e category

test[(test.pred==test.pred_pred) &\
             (test.pred_pred.isin(['yes','no']))]\
             .groupby('label').size() / test[(test.pred==test.pred_pred)].groupby('label').size() 

label
5    0.3
dtype: float64

In [37]:
# Same for lab1
lab1.exp_enc = lab1.exp_enc.apply(pad_exp)
lab1.qstr_enc = lab1.qstr_enc.apply(pad_exp)

lab1_predict = model.predict([np.vstack(lab1['qstr_enc'].to_numpy()),
          np.vstack(lab1['exp_enc'].to_numpy())])
lab1['pred_pred'] = enc.inverse_transform(lab1_predict)

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [38]:
test['label'] = 0

finaltest = pd.concat([lab1,test])
finaltest['pred_label'] = (finaltest.pred_pred != finaltest.pred) #model couldnt predict true ans --> error detected
finaltest['label'] = finaltest['label'] == 1 #True == A-E error detected by researcher
pred_label = finaltest['pred_label'].to_numpy().reshape(-1,1)
true_label = finaltest['label'].to_numpy().reshape(-1,1)

print(classification_report(pred_label,true_label))
np.hstack((pred_label,true_label))


              precision    recall  f1-score   support

       False       0.73      0.80      0.76       188
        True       0.59      0.49      0.53       110

    accuracy                           0.68       298
   macro avg       0.66      0.64      0.65       298
weighted avg       0.68      0.68      0.68       298



array([[False,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ Tr

In [353]:

len(lab1), len(test)

(92, 207)